In [1]:
import sys
sys.path.append('/Users/bjg/MovieRecommender_AI')

from Modeling.collaborative_filtering import *
from Modeling.content_filtering import *
from Modeling.graphing import *
from surprise import SVD
from surprise.model_selection import cross_validate, train_test_split
from surprise import Dataset
from surprise import Reader

In [2]:
movies_metadata_df = pd.read_csv('../data/the-movies-dataset/movies_metadata.csv'
                                 , converters={ 'id': lambda x: convert_ids(x)
                                               , 'imdb_id': lambda x: convert_ids(x)
                                               ,'popularity': lambda x: convert_to_float(x)
                                               ,'genres': lambda x: to_json(x)}
                                 , usecols=['id', 'original_title'
                                                , 'genres' #'homepage'
                                                , 'overview', 'popularity', 'poster_path'
                                                , 'release_date', 'revenue', 'runtime'
                                                , 'spoken_languages', 'title'
                                                , 'vote_average', 'vote_count']
                                , dtype={'populariy': np.float64}
                                , parse_dates=True)


movies_lookup_df = pd.read_csv('../data/the-movies-dataset/movies_metadata.csv'
                        , converters={'id': lambda x: convert_ids(x), 'imdb_id': lambda x: convert_ids(x)}
                       ,usecols=['id', 'title'])
#####################################
##SVD DATA SET
movies_df = pd.read_csv('../data/the-movies-dataset/movies_metadata.csv'
                        , converters={'id': lambda x: convert_ids(x), 'imdb_id': lambda x: convert_ids(x)}
                       ,usecols=['id', 'original_title', 'belongs_to_collection'
                                 , 'budget', 'genres', 'homepage'
                                 ,'imdb_id', 'overview', 'popularity', 'poster_path'
                                 , 'production_companies','release_date', 'revenue', 'runtime',
                                 'spoken_languages', 'status', 'tagline', 'title', 'video',
                                 'vote_average', 'vote_count'])
#####################################

ratings_df = pd.read_csv('../data/the-movies-dataset/ratings_small.csv')



content_filter_df = pd.read_pickle('../data/content_filter_df.pkl')
content_filter_df = content_filter_df[['id',
 'popularity',
 #'release_date',
 'vote_average',
 'release_year',
 0,1,2,3,4,5, 6, 7, 8,9,10,11,12,13,14,15,16,17,18,19]]
content_filter_df = content_filter_df.dropna()

idx = pd.Index(content_filter_df['id'])
idx
content_filter_df.index = idx

/anaconda3/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3020: DtypeWarning: Columns (10) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [3]:
###May need Fuzzy matching, but for now:
movies_df = movies_df[movies_df.spoken_languages == """[{'iso_639_1': 'en', 'name': 'English'}]"""]
ratings_with_movie_names = ratings_df.merge(movies_df[['id', 'original_title']], how='left', left_on='movieId', right_on='id')
ratings_with_movie_names = ratings_with_movie_names[ratings_with_movie_names.original_title.isnull() == False]

In [4]:
reader = Reader(rating_scale=(0, 5))
data = Dataset.load_from_df(ratings_with_movie_names[['userId', 'movieId', 'rating']], reader)
trainset = data.build_full_trainset()
testset = trainset.build_anti_testset()

In [5]:
algo = SVD(verbose=True, random_state=51)
algo.fit(trainset)

cross_validate(algo, data, measures=['RMSE', 'MAE'], cv=5, n_jobs=-1)#, verbose=True)

Processing epoch 0
Processing epoch 1
Processing epoch 2
Processing epoch 3
Processing epoch 4
Processing epoch 5
Processing epoch 6
Processing epoch 7
Processing epoch 8
Processing epoch 9
Processing epoch 10
Processing epoch 11
Processing epoch 12
Processing epoch 13
Processing epoch 14
Processing epoch 15
Processing epoch 16
Processing epoch 17
Processing epoch 18
Processing epoch 19


{'test_rmse': array([0.88071965, 0.89998092, 0.90418479, 0.90266606, 0.88545311]),
 'test_mae': array([0.68448699, 0.68377255, 0.69980006, 0.69972166, 0.68481808]),
 'fit_time': (0.8804490566253662,
  0.8936948776245117,
  0.9232771396636963,
  0.9181864261627197,
  0.8151097297668457),
 'test_time': (0.028622150421142578,
  0.026682138442993164,
  0.030500173568725586,
  0.023614168167114258,
  0.022730350494384766)}

In [7]:
predictions = algo.test(testset)
### Tune this value to get fewer results faster, but less options to choose from
top_n = get_top_n(predictions, 200)


predicted_movies_by_name = defaultdict(list)

### This builds the dictionary of predicted movies for all users
for key, value in top_n.items():
    predicted_movies_by_name[key] = [get_movie_name(mov_id[0], movies_metadata_df) for mov_id in value]

**SVD IS BOOTSTRAPPED**

In [99]:
selected_user = 338
_collab_recs_ = sorted(users_favorite_movies(10, selected_user, predicted_movies_by_name, movies_metadata_df, ratings_with_movie_names), key=lambda k: k[1], reverse=True)
top_three = _collab_recs_[0:3]
top_three

[UserFavoriteRating(title='48 Hrs.', rating=4.0),
 UserFavoriteRating(title='Stand by Me', rating=4.0),
 UserFavoriteRating(title='The Million Dollar Hotel', rating=4.0)]

In [107]:
collab_recs = collab_filter_recommendations(selected_user, top_n, movies_metadata_df)
collab_recs[0:30]

[UserFavoriteRating(title='Birdman of Alcatraz', rating=4.157525275035756),
 UserFavoriteRating(title='My Darling Clementine', rating=4.1564348326804135),
 UserFavoriteRating(title='While You Were Sleeping', rating=4.143476069427995),
 UserFavoriteRating(title='The Rapture', rating=4.074503797480896),
 UserFavoriteRating(title='Lonely Hearts', rating=4.055506019538544),
 UserFavoriteRating(title='Men in Black II', rating=4.034652786292273),
 UserFavoriteRating(title='Hard Target', rating=4.023421276458529),
 UserFavoriteRating(title='Shriek If You Know What I Did Last Friday the Thirteenth', rating=4.022937473968079),
 UserFavoriteRating(title='Nell', rating=4.019701479404814),
 UserFavoriteRating(title='The Good Thief', rating=4.014500919573835),
 UserFavoriteRating(title='Dog Day Afternoon', rating=3.9981395225115595),
 UserFavoriteRating(title='Flags of Our Fathers', rating=3.9940314992587034),
 UserFavoriteRating(title='Galaxy Quest', rating=3.9812950848530377),
 UserFavoriteRating

In [72]:
number_of_similar_films = 250

fav_movies = [mov[0] for mov in top_three]

origin_film_1 = get_movie_id(fav_movies[0], movies_metadata_df)

similar_films_scored_1 = top_n_closest_content_filtered(number_of_similar_films, origin_film_1, content_filter_df)
content_filter_recs_1 = top_filtered_to_movie_rec_tuples(similar_films_scored_1, origin_film_1, movies_lookup_df)

origin_film_2 = get_movie_id(fav_movies[1], movies_metadata_df)

similar_films_scored_2 = top_n_closest_content_filtered(number_of_similar_films, origin_film_2, content_filter_df)
content_filter_recs_2 = top_filtered_to_movie_rec_tuples(similar_films_scored_2, origin_film_2, movies_lookup_df)

origin_film_3 = get_movie_id(fav_movies[2], movies_metadata_df)

similar_films_scored_3 = top_n_closest_content_filtered(number_of_similar_films, origin_film_3, content_filter_df)
content_filter_recs_3 = top_filtered_to_movie_rec_tuples(similar_films_scored_3, origin_film_3, movies_lookup_df)

lookup failed 143 times.
lookup failed 144 times.
lookup failed 138 times.


In [101]:
content_filter_recs_3[0:10]

[ContentFilterRecommendation(id_origin=318, title='The Million Dollar Hotel', id_2=6609, title_2='Keoma', similarity=0.9999984682726184),
 ContentFilterRecommendation(id_origin=318, title='The Million Dollar Hotel', id_2=13305, title_2='Snow-White and Rose-Red', similarity=0.9999971597777696),
 ContentFilterRecommendation(id_origin=318, title='The Million Dollar Hotel', id_2=1777, title_2='Fahrenheit 9/11', similarity=0.9999951665612803),
 ContentFilterRecommendation(id_origin=318, title='The Million Dollar Hotel', id_2=11109, title_2='Vera Drake', similarity=0.9999932014251952),
 ContentFilterRecommendation(id_origin=318, title='The Million Dollar Hotel', id_2=896, title_2='The World of Apu', similarity=0.9999903560191948),
 ContentFilterRecommendation(id_origin=318, title='The Million Dollar Hotel', id_2=11403, title_2='The River Why', similarity=0.9999901600742706),
 ContentFilterRecommendation(id_origin=318, title='The Million Dollar Hotel', id_2=2239, title_2='Hallam Foe', similar

In [103]:
content_filter_recs_2[0:10]

[ContentFilterRecommendation(id_origin=235, title='Stand by Me', id_2=284, title_2='The Apartment', similarity=0.9999942452745557),
 ContentFilterRecommendation(id_origin=235, title='Stand by Me', id_2=1907, title_2='The Beach', similarity=0.9999939908238119),
 ContentFilterRecommendation(id_origin=235, title='Stand by Me', id_2=1285, title_2='Torrente 3 The Protector', similarity=0.999991693236627),
 ContentFilterRecommendation(id_origin=235, title='Stand by Me', id_2=876, title_2="Frank Herbert's Dune", similarity=0.9999916416373108),
 ContentFilterRecommendation(id_origin=235, title='Stand by Me', id_2=337, title_2='Monsieur Ibrahim', similarity=0.9999869610449237),
 ContentFilterRecommendation(id_origin=235, title='Stand by Me', id_2=10527, title_2='Madagascar: Escape 2 Africa', similarity=0.9999833398879097),
 ContentFilterRecommendation(id_origin=235, title='Stand by Me', id_2=1998, title_2='Holy Lola', similarity=0.9999827025003529),
 ContentFilterRecommendation(id_origin=235, t

In [102]:
content_filter_recs_1[0:10]

[ContentFilterRecommendation(id_origin=150, title='48 Hrs.', id_2=4909, title_2='The Day of the Jackal', similarity=0.9999999999999999),
 ContentFilterRecommendation(id_origin=150, title='48 Hrs.', id_2=1811, title_2='Nowhere', similarity=0.9999981612780758),
 ContentFilterRecommendation(id_origin=150, title='48 Hrs.', id_2=11142, title_2='Der Clown - Tag der Vergeltung', similarity=0.9999971228637452),
 ContentFilterRecommendation(id_origin=150, title='48 Hrs.', id_2=2608, title_2='Maid to Order', similarity=0.9999912344970128),
 ContentFilterRecommendation(id_origin=150, title='48 Hrs.', id_2=4823, title_2='Final Analysis', similarity=0.9999869167597799),
 ContentFilterRecommendation(id_origin=150, title='48 Hrs.', id_2=10439, title_2='Hocus Pocus', similarity=0.9999861673058159),
 ContentFilterRecommendation(id_origin=150, title='48 Hrs.', id_2=829, title_2='Chinatown', similarity=0.9999856844441974),
 ContentFilterRecommendation(id_origin=150, title='48 Hrs.', id_2=1213, title_2='T

In [76]:
uri = "bolt://localhost:7687"
driver = GraphDatabase.driver(uri, auth=('neo4j', 'n;slr#!21'))

In [77]:
first_degree_away_films = get_connected_movies([mov[0] for mov in top_three], driver)

In [78]:
graph_name_collab_filter = set({movie[0] for movie in first_degree_away_films})

In [91]:
content_filter_recs = content_filter_recs_1 + content_filter_recs_2 + content_filter_recs_3

In [92]:
content_rec_names_set = set(movie[3] for movie in content_filter_recs)

In [93]:
collab_movie_name_set = set(movie[0] for movie in collab_recs)

In [94]:
graph_name_collab_filter & content_rec_names_set

{'A Nightmare on Elm Street 3: Dream Warriors',
 'Across the Universe',
 'Armageddon',
 'Babel',
 'Cat People',
 'Crash',
 'Dark Blue',
 'Ed Wood',
 'Fight Club',
 'Final Analysis',
 'Funny Games',
 'Planet of the Apes',
 'Poodle Springs',
 'Star Wars: Episode II - Attack of the Clones',
 'Star Wars: Episode III - Revenge of the Sith',
 'The Apostle',
 'The Golden Child',
 'The Legend of 1900'}

In [95]:
graph_name_collab_filter & collab_movie_name_set

{'1408',
 'Across the Universe',
 'American Graffiti',
 'Armed and Dangerous',
 'Bean',
 'Beverly Hills Cop III',
 'Blown Away',
 'Dancer in the Dark',
 'Ghost Rider',
 'In the Mouth of Madness',
 'Ken Park',
 'Lethal Weapon 3',
 'Nell',
 "One Night at McCool's",
 'The Final Cut',
 'The Good Thief',
 'The Number 23',
 'Twin Peaks: Fire Walk with Me',
 'Wedlock'}

In [96]:
recommendation_titles = (graph_name_collab_filter & content_rec_names_set) | (graph_name_collab_filter & collab_movie_name_set)

In [97]:
recommendation_titles

{'1408',
 'A Nightmare on Elm Street 3: Dream Warriors',
 'Across the Universe',
 'American Graffiti',
 'Armageddon',
 'Armed and Dangerous',
 'Babel',
 'Bean',
 'Beverly Hills Cop III',
 'Blown Away',
 'Cat People',
 'Crash',
 'Dancer in the Dark',
 'Dark Blue',
 'Ed Wood',
 'Fight Club',
 'Final Analysis',
 'Funny Games',
 'Ghost Rider',
 'In the Mouth of Madness',
 'Ken Park',
 'Lethal Weapon 3',
 'Nell',
 "One Night at McCool's",
 'Planet of the Apes',
 'Poodle Springs',
 'Star Wars: Episode II - Attack of the Clones',
 'Star Wars: Episode III - Revenge of the Sith',
 'The Apostle',
 'The Final Cut',
 'The Golden Child',
 'The Good Thief',
 'The Legend of 1900',
 'The Number 23',
 'Twin Peaks: Fire Walk with Me',
 'Wedlock'}

In [98]:
for mov in collab_recs:
    if mov[0] in recommendation_titles:
        print(mov)

UserFavoriteRating(title='Nell', rating=4.019701479404814)
UserFavoriteRating(title='The Good Thief', rating=4.014500919573835)
UserFavoriteRating(title='Beverly Hills Cop III', rating=3.9677392479904574)
UserFavoriteRating(title='Ghost Rider', rating=3.8672282395156294)
UserFavoriteRating(title="One Night at McCool's", rating=3.772803782922307)
UserFavoriteRating(title='Bean', rating=3.759164995300022)
UserFavoriteRating(title='Wedlock', rating=3.720574379857845)
UserFavoriteRating(title='American Graffiti', rating=3.712864864203836)
UserFavoriteRating(title='Ken Park', rating=3.6609258995186735)
UserFavoriteRating(title='Armed and Dangerous', rating=3.6428484979952565)
UserFavoriteRating(title='Blown Away', rating=3.6321901223846957)
UserFavoriteRating(title='The Number 23', rating=3.625648055487206)
UserFavoriteRating(title='Lethal Weapon 3', rating=3.624913736497648)
UserFavoriteRating(title='1408', rating=3.583086092974816)
UserFavoriteRating(title='In the Mouth of Madness', ratin